In [21]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans

In [22]:
nba_data = pd.read_csv('Data/nba_data.csv')
players = nba_data.player.to_list()


In [23]:
nba_data = nba_data.set_index('rank')
nba_data.drop(['Unnamed: 0', 'player', 'pos', 'team_id', 'age', 'mp', 'fg', 'fga','fg3',
 'fg3a','fg2',
 'fg2a','ft',
 'fta','orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',], axis=1, inplace=True)
list(nba_data.columns.values)


['g',
 'gs',
 'fg_pct',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fg_per_mp',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fg_per_poss',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',
 'off_rtg',
 'def_rtg',
 'per',
 'ts_pct',
 'fg3a_per_fga_pct',
 'fta_per_fga_pct',
 'orb_pct',
 'drb_pct',
 'trb_pct',
 'ast_pct',
 'stl

In [24]:
nba_data

,g,gs,fg_pct,fg3_pct,fg2_pct,efg_pct,ft_pct,mp_per_g,fg_per_g,fga_per_g,...,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
rank,,,,,,,,,,,,,,,,,,,,,
1,31,2,0.357,0.323,0.500,0.487,0.923,19.0,1.8,5.1,...,7.9,12.2,0.1,0.6,0.6,0.053,-2.4,-0.9,-3.4,-0.2
3,34,1,0.345,0.338,0.361,0.459,0.778,12.6,1.1,3.2,...,19.7,13.5,-0.1,0.2,0.1,0.011,-3.1,-1.3,-4.4,-0.3
4,80,80,0.595,0.000,0.596,0.595,0.500,33.4,6.0,10.1,...,12.6,16.4,5.1,4.0,9.1,0.163,0.6,2.1,2.7,3.2
5,82,28,0.576,0.200,0.588,0.579,0.735,23.3,3.4,5.9,...,17.1,15.8,3.4,3.4,6.8,0.171,-0.6,3.6,3.0,2.4
8,81,81,0.519,0.238,0.528,0.522,0.847,33.2,8.4,16.3,...,8.8,26.9,6.4,2.9,9.3,0.167,1.2,0.5,1.6,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,75,55,0.422,0.341,0.515,0.513,0.748,27.2,2.8,6.6,...,11.5,13.2,0.3,2.0,2.2,0.052,-1.7,0.6,-1.2,0.4
488,49,27,0.559,0.292,0.586,0.572,0.795,22.3,6.1,11.0,...,12.7,28.5,2.2,1.9,4.2,0.183,0.1,1.2,1.2,0.9
521,21,2,0.521,0.346,0.571,0.560,0.732,12.0,2.9,5.6,...,10.7,26.1,0.4,0.3,0.8,0.151,-1.5,-1.8,-3.3,-0.1


In [28]:
pd.options.display.max_rows = 100

def run_cluster(data, num_clusters=5):
    # create kmeans model
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)

    # find centers and labels
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_

    # DataFrame of centroids
    centroids_df = pd.DataFrame(centers, columns=data.columns).T

    # print items in clusters
    clusters = {}
    for i in range(len(labels)):
        clusters[players[i]] = labels[i]

    for i in range(len(centers)):
        out = [k for k,v in clusters.items() if v==i]

        print('-'*30)
        print(f'cluster index: {i}')
        for item in out:
            print(item)
        print('\n')

    centroids_df.to_csv('final_clusters.csv')
    return centroids_df

In [29]:
run_cluster(nba_data, 8)

------------------------------
cluster index: 0
Grayson Allen
Justin Anderson
Dwayne Bacon
J.J. Barea
Jerryd Bayless
Dragan Bender
Antonio Blakeney
Jonah Bolden
Isaiah Briscoe
Ryan Broekhoff
Sterling Brown
Troy Brown
Jose Calderon
Jevon Carter
Gary Clark
Ian Clark
Allen Crabbe
Jamal Crawford
Troy Daniels
Hamidou Diallo
Dante Exum
Devonte' Graham
Jerian Grant
Solomon Hill
Aaron Holiday
Chandler Hutchison
Amir Johnson
Frank Kaminsky
Furkan Korkmaz
Luke Kornet
Shaun Livingston
Jarell Martin
Frank Mason
Abdel Nader
Shabazz Napier
Raul Neto
Georges Niang
Dirk Nowitzki
Frank Ntilikina
David Nwaba
Semi Ojeleye
Elie Okobo
Tony Parker
Patrick Patterson
Quincy Pondexter
Norman Powell
Chasson Randle
Glenn Robinson
Thabo Sefolosha
Ish Smith
Omari Spellman
Lance Thomas
Sindarius Thornwell
Anthony Tolliver
Moritz Wagner
Tyrone Wallace
D.J. Wilson
Marquese Chriss
Sam Dekker
Matthew Dellavedova
Tyler Dorsey
Tim Frazier
Wesley Johnson
Brandon Knight
Timothe Luwawu-Cabarrot
Shelvin Mack
Thon Maker
C.J. 

,0,1,2,3,4,5,6,7
g,49.690141,73.250000,74.213115,63.489362,48.046512,72.316456,73.750000,27.067797
gs,6.154930,67.821429,65.426230,9.553191,36.000000,13.784810,73.694444,3.474576
fg_pct,0.412155,0.552286,0.440279,0.546191,0.446814,0.436506,0.465361,0.438932
fg3_pct,0.316549,0.243321,0.363328,0.209213,0.310930,0.347177,0.345167,0.288576
fg2_pct,0.481437,0.581214,0.500016,0.576617,0.492791,0.505785,0.515861,0.495153
efg_pct,0.486901,0.573643,0.522475,0.569681,0.505209,0.517481,0.529000,0.500102
ft_pct,0.732000,0.718250,0.795770,0.698043,0.742907,0.772810,0.815167,0.716780
mp_per_g,15.123944,29.057143,29.018033,16.727660,26.672093,21.981013,33.450000,14.527119
fg_per_g,1.995775,6.253571,4.311475,2.740426,4.567442,3.336709,7.955556,1.928814
fga_per_g,4.871831,11.507143,9.808197,5.027660,10.248837,7.655696,17.130556,4.430508
